### 필요한 library install

In [ ]:
! pip install category_encoders
! pip install pycaret
! pip install mljar-supervised
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

### import library

In [ ]:
import datetime as dt
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from functools import reduce
import pickle
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
plt.rc('font', family='NanumBarunGothic')

### Setting seed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(23)

### Load Data

In [ ]:
path = r"/content/drive/MyDrive/data_analysis/대구 교통사고 피해 예측 AI 경진대회/data"
fpath = r"/content/drive/MyDrive/data_analysis/대구 교통사고 피해 예측 AI 경진대회/data/external_open"

In [ ]:
# 데이터 불러오기 (외부데이터 포함)
train_data = pd.read_csv(path + "/train.csv")
test_data = pd.read_csv(path + "/test.csv")
acc = pd.read_csv(fpath + "/countrywide_accident.csv")
CCTV = pd.read_csv(fpath + "/대구 CCTV 정보.csv", encoding = 'cp949')
light = pd.read_csv(fpath + "/대구 보안등 정보.csv", encoding = 'cp949', low_memory=False)
child_protect = pd.read_csv(fpath + "/대구 어린이 보호 구역 정보.csv", encoding = 'cp949')
parking = pd.read_csv(fpath + "/대구 주차장 정보.csv", encoding = 'cp949')

In [ ]:
# 제출 파일 불러오기
sample_submission = pd.read_csv(path + '/submission.csv', encoding = 'cp949')
baseline_submission = sample_submission.copy()

In [ ]:
train = train_data.copy()
test = test_data.copy()

In [ ]:
# 우선 test data의 column과 맞춰주기
cols = ['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', 'ECLO']
train = train[cols]

In [ ]:
# 시공간 파생변수 추출
train['사고일시'] = train['사고일시'].astype('datetime64[ns]')
train['사고월'] = train['사고일시'].dt.month.astype(str)
train['사고일'] = train['사고일시'].dt.day.astype(str)
train['사고시간'] = train['사고일시'].dt.hour.astype(str)

location_pattern = r'(\S+) (\S+) (\S+)'
train[['도시', '구', '동']] = train['시군구'].str.extract(location_pattern)
train = train.drop(columns=['시군구'])

In [ ]:
test['사고일시'] = test['사고일시'].astype('datetime64[ns]')
test['사고월'] = test['사고일시'].dt.month.astype(str)
test['사고일'] = test['사고일시'].dt.day.astype(str)
test['사고시간'] = test['사고일시'].dt.hour.astype(str)

location_pattern = r'(\S+) (\S+) (\S+)'
test[['도시', '구', '동']] = test['시군구'].str.extract(location_pattern)
test = test.drop(columns=['시군구'])

In [ ]:
def process_location_data(df, address_column, drop_columns):
    location_pattern = r'(\S+) (\S+) (\S+) (\S+)'
    df[['도시', '구', '동', '우편번호']] = df[address_column].str.extract(location_pattern)
    df = df.drop(drop_columns, axis=1)
    return df

In [ ]:
CCTV = process_location_data(CCTV, '소재지지번주소', ['소재지지번주소', '우편번호'])
light = process_location_data(light, '소재지지번주소', ['소재지지번주소', '우편번호'])
parking = process_location_data(parking, '소재지지번주소', ['소재지지번주소', '우편번호'])
child_protect = process_location_data(child_protect, '소재지지번주소', ['소재지지번주소', '우편번호'])

CCTV_notnull = CCTV.dropna(subset=['동'])
light_notnull = light.dropna(subset=['동'])
parking_notnull = parking.dropna(subset=['동'])
child_protect_notnull = child_protect.dropna(subset=['동'])

In [ ]:
# CCTV의 경우, 개별 정보이기 때문에 동별로 개수 count
CCTV_num = CCTV_notnull.groupby('동').size().reset_index(name='CCTV개수')

# 보안등의 경우, 동별로 설치 개수가 나타나있기 때문에 count
light_num = light_notnull.groupby('동')['설치개수'].sum()
light_num = pd.DataFrame(light_num).reset_index()
light_num.rename(columns = {'설치개수' : '보안등개수'}, inplace = True)

# 주차장 일단 급지지역으로 one-hot encoding
parking_cat = pd.get_dummies(parking_notnull[['동', '급지구분']], columns=['급지구분'], prefix='급지')
parking_num = parking_notnull.groupby('동').size()
parking_num = pd.DataFrame(parking_num).reset_index()
# 그리고 주차장 개수 더하기
parking_num.rename(columns = {0 : '주차장개수'}, inplace = True)
parking_pre = pd.merge(parking_num, parking_cat, how = 'left', on = '동')
parking_pre = parking_pre.groupby('동').sum().reset_index()

# 동 안에 초등학교 개수 세기
elementary_num = child_protect_notnull.groupby('동')['대상시설명'].nunique()
elementary_num = pd.DataFrame(elementary_num).reset_index()
elementary_num.rename(columns = {'대상시설명' : '초등학교개수'}, inplace = True)

In [ ]:
# train, test data + 외부데이터
join_1 = pd.merge(train, CCTV_num, on = '동', how = 'left')
join_2 = pd.merge(join_1, light_num, on = '동', how = 'left')
join_3 = pd.merge(join_2, parking_pre, on = '동', how = 'left')
join_4 = pd.merge(join_3, elementary_num, on = '동', how = 'left')
train_raw = join_4.fillna(0).copy()

join_1 = pd.merge(test, CCTV_num, on = '동', how = 'left')
join_2 = pd.merge(join_1, light_num, on = '동', how = 'left')
join_3 = pd.merge(join_2, parking_pre, on = '동', how = 'left')
join_4 = pd.merge(join_3, elementary_num, on = '동', how = 'left')
test_raw = join_4.fillna(0).copy()

In [ ]:
# 이상치 제거
for i in ['주차장개수', '보안등개수']:
  Q1 = train_raw[i].quantile(0.25)
  Q3 = train_raw[i].quantile(0.75)
  IQR = Q3 - Q1

  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  train_raw[i] = train_raw[i].clip(lower=lower_bound, upper=upper_bound)

In [ ]:
# holiday
holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']

In [ ]:
# 파생변수 생성
train_raw['사고일시'] = pd.to_datetime(train_raw['사고일시'])
train_raw['day_of_week'] = train_raw['사고일시'].dt.dayofweek
train_raw['holiday'] = np.where((train_raw.day_of_week >= 5) | (train_raw.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

test_raw['사고일시'] = pd.to_datetime(test_raw['사고일시'])
test_raw['day_of_week'] = test_raw['사고일시'].dt.dayofweek
test_raw['holiday'] = np.where((test_raw.day_of_week >= 5) | (test_raw.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [ ]:
train_raw['사고시간'] = train_raw['사고시간'].astype('int')
test_raw['사고시간'] = test_raw['사고시간'].astype('int')

train_raw['Cosine_Time'] = np.cos(2 * np.pi * train_raw['사고시간'] / 24)
test_raw['Cosine_Time'] = np.cos(2 * np.pi * test_raw['사고시간'] / 24)

In [ ]:
def group_season(df):
    df.loc[(df['사고월'] == '3') | (df['사고월'] == '4') | (df['사고월'] == '5'), 'season'] = '봄'
    df.loc[(df['사고월'] == '6') | (df['사고월'] == '7') | (df['사고월'] == '8'), 'season'] = '여름'
    df.loc[(df['사고월'] == '9') | (df['사고월'] == '10') | (df['사고월'] == '11'), 'season'] = '가을'
    df.loc[(df['사고월'] == '12') | (df['사고월'] == '1') | (df['사고월'] == '2'), 'season'] = '겨울'
    return df['season']

train_raw['season'] = group_season(train_raw)
test_raw['season'] = group_season(test_raw)

In [ ]:
acc_cnt = train_raw.groupby('동').size().reset_index(name='사고횟수')
train_raw = pd.merge(train_raw, acc_cnt, on = '동', how = 'left')
test_raw = pd.merge(test_raw, acc_cnt, on = '동', how = 'left')

In [ ]:
train_raw['요일'] = ((train_raw['요일'] == '토요일')|(train_raw['요일'] == '일요일')).astype(int)
train_raw['기상상태'] = (train_raw['기상상태'] == '안개').astype(int)
train_raw['사고유형'] = (train_raw['사고유형'] == '차대차').astype(int)

test_raw['요일'] = ((test_raw['요일'] == '토요일')|(test_raw['요일'] == '일요일')).astype(int)
test_raw['기상상태'] = (test_raw['기상상태'] == '안개').astype(int)
test_raw['사고유형'] = (test_raw['사고유형'] == '차대차').astype(int)

In [ ]:
# Target Encoding
for i in ['도로형태', '노면상태', '동', 'season']:
    le = ce.TargetEncoder(cols=[i])
    train_raw[i] = le.fit_transform(train_raw[i], train_raw['ECLO'])
    test_raw[i] = le.transform(test_raw[i])

In [ ]:
train_raw = train_raw.drop(['ID', '사고일시', '도시', '구'], axis = 1)
test_raw = test_raw.drop(['ID', '사고일시', '도시', '구'], axis = 1)

In [ ]:
# data Scaling
scaler = StandardScaler()
scale_col = train_raw.drop('ECLO', axis =1).columns
train_raw[scale_col] = scaler.fit_transform(train_raw[scale_col])
test_raw[scale_col] = scaler.fit_transform(test_raw[scale_col])

### Prediction with AutoML

In [ ]:
from supervised.automl import AutoML

In [ ]:
automl = AutoML(mode="Compete",
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
                n_jobs = -1,total_time_limit=43200, eval_metric="rmse", ml_task = "regression",)

In [ ]:
automl.fit(X, y) # 약 4시간 소요

In [ ]:
pred = automl.predict(test_raw)
baseline_submission['ECLO'] = pred

In [ ]:
baseline_submission.to_csv(path + '/mljar2_submission.csv', index = False) # 0.42785